In [326]:
import numpy as np
import pandas as pd

In [335]:
data = pd.read_csv('train.csv')
submit = pd.DataFrame()
submit['id'] = data.id
submit['nextvisit'] = 0

In [332]:
alpha = 1.2
all_weights = sum([x ** alpha for x in range(1, 158)])

def get_probs(weeks, days, alpha=1.2):
    probs_by_day = []
    probs_result = []
    
    #all_weights = math.log(reduce(lambda x, y: x*y, weeks))
    # all_weights = sum([x ** alpha for x in weeks])
    for day in range(1, 8):
        weight_sum = 0
        for i, val in enumerate(weeks):
            if(days[i] == day):
                weight_sum += weeks[i] ** alpha
        probs_by_day.append(weight_sum / all_weights)
        
#     for i in range(len(probs_by_day)):
#         multiply = 1
#         if(i > 0):
#             for k in range(i):
#                 multiply *= (1 - probs_by_day[k])
#         probs_result.append(probs_by_day[i] * multiply)
    
    return probs_by_day, probs_result

### Пересчет вероятностей с весами 

In [333]:
alpha = 1.2

In [334]:
import math

def get_predicted_day(user_id):
    visits = data[user_id:user_id+1].visits.values[0].split()
    weeks = [int(x) / 7 if (int(x) % 7) == 0 else int(x) / 7 + 1 for x in visits]
    days = [7 if (int(x) % 7 == 0) else (int(x) % 7) for x in visits]
    
    probs_by_day, probs_result = get_probs(weeks, days)        
    
    return probs_by_day.index(max(probs_by_day)) + 1

In [336]:
for i in range(len(data)):
    submit.loc[submit['id'] == i + 1, 'nextvisit'] = get_predicted_day(i)

In [339]:
submit[:10]

,id,nextvisit
0,1,2
1,2,3
2,3,6
3,4,5
4,5,2
5,6,3
6,7,4
7,8,3
8,9,6
9,10,6


### Linear SVC

In [337]:
submFile = open('submit.csv', 'w')
submFile.write('id,nextvisit\n')
for i in range(len(submit)):
    submFile.write(str(submit['id'].values[i]))
    submFile.write(', ')
    submFile.write(str(submit['nextvisit'].values[i]))
    submFile.write('\n')

submFile.close()